# Task 2: a more challenging percolation problem

In this notebook, we are going to simulate a percolating substance that is able to propagate horizontally, as well as downwards.

As a physical motivation, we might think of the percolating substance as water, and the simulation as representing water percolating through soil under gravity.
This is actually a very relevant issue today; for example, understanding the way in which water percolates through different substances is crucial for accurate flood risk assessments, and flood mitigation strategies often aim to slow down the speed of percolation, without stopping it (which would lead to surface runoff and make the situation worse).

The aim of this notebook will be to determine the value of $q$ at which percolation switches from becoming more than 50\% probable, to less than 50\% probable.

<div class="alert alert-block alert-success">
    <b>Definition:</b> We will define the <b>percolation threshold</b> $q^*$ as the value of $q$ which corresponds to a percolation probability of $p = \frac{1}{2}$.
</div>

## Running this variation of the model

### Importing the code

Of course, the first thing we need to do is import the relevant bits of code by running the next cell.

In [ ]:
from percolation.lattice import SquareLattice
from percolation.model import PercolationModel
from percolation.scripts.parameter_scan import parameter_scan

### Setting up the model

We're going to provide a second argument to `SquareLattice`, called `n_links`.
In the previous notebook, `n_links` took its default value of 1, but we now want to change it to 3.

Run the following cell to see the effect of changing `n_links`.

In [ ]:
help(SquareLattice.n_links)

The way we will create a $50\times 50$ lattice is
```python
lattice = SquareLattice(50, 50, n_links=3)
```

You might wonder why we don't just write
```python
lattice = SquareLattice(50, 50, 3)
```
Python allows both, but it is usually preferable to explicitly state the name of the argument which you are providing a value for.
This is particularly helpful when a function has many arguments, and you don't want to rely on remembering the order in which they are supposed to appear.

We could have been even more explicit and written
```python
lattice = SquareLattice(n_rows=50, n_cols=50, n_links=3)
```
but it's easy to remember that the first two arguments specify the size of the lattice.

Run the cell below to produce an animation of this new variant of the model.

In [ ]:
q = 0.08

lattice = SquareLattice(50, 50, n_links=3)
model = PercolationModel(lattice, q)

model.animate(60)

Notice that the percolation transition (when the water reaches the bottom row in the simulation) is surely going to be found at higher $q$ than in the previous notebook, since it doesn't only travel in one direction.

In the cell below, try a few different values of $q$, to get a rough idea of approximately where the percolation threshold lies (i.e. where the probability of percolating is $1/2$).

In [ ]:
q = 0.3  # change me!

lattice = SquareLattice(100, 50, n_links=3)
model = PercolationModel(lattice, q)

model.animate(200)

## The percolation transition

To estimate the percolation probability $p$, we will use a method similar to one you saw in the previous notebook.

For each value of $q$ (the probability that a node is inert) we will run a number, $N$, of separate simulations and record the fraction of these that percolated,

$$
f = \frac{1}{N} \sum_{i=1}^N b_i \tag{Estimator}
$$

where $b_i$ are equal to either 0 (for simulations which don't percolate) or 1 (for those that do), and $N$ is the number of simulations.

The estimator for the standard error is the same as before:

$$
\sigma_f = \sqrt{\frac{f(1 - f)}{N-1}} \tag{Standard error estimator}
$$

In the example with light, we were able to *calculate* the true probability $p$, and compare this with our estimate.
Here, we cannot do that (at least, I don't know how to calculate it...)

Instead, we will be forced to draw conclusions based on the results of our simulations, and nothing else!
This is why it was so valuable to check that everything was working properly in the previous notebook, when we knew the answer a priori.

## Estimating the percolation threshold using a 'fit'

We want to find the percolation threshold, $q^*$, that gives a percolation probability of $p = 1/2$.
To do this we will attempt to **fit** a function to the data, and use the **best-fit values of the parameters** of the function to estimate $q^*$.

This may seem rather scary, but actually this approach might be familiar to you from school; you are probably quite familiar with a **line of best fit**, which corresponds to the function $f(x) = m x + c$, whose parameters are $m$ (the gradient) and $c$ (the y-intercept).
The purpose of plotting a line of best fit is to extract the best-fit values of the gradient and the y-intercept, which might contain useful information about the system.
This case is not different, other than that we will use a different function with different parameters.

<div class="alert alert-block alert-success">
    <b>Definition:</b> A <b>fit</b> is when you take a function and carefully tune the parameters so that the resulting line matches your data as well as possible.
</div>

Interested students can take 5 minutes at this point to have a read through the section [Further reading: Fitting functions to data](#Further-reading:-Fitting-functions-to-data) before returning to this point.

The function we will attempt to fit to our data is a **Logistic** function

$$
S(q) = \frac{1}{1 + e^{\lambda (q - q_0)}}
$$

$S(q)$ is a very simple function that smoothly transitions between the asymptotes

$$
\lim_{q\to-\infty} S(q) = 1 \qquad\qquad
\lim_{q\to\infty} S(q) = 0
$$

and has two *parameters* (just like the line of best fit):
* The **mid-point** $q_0$, such that $S(q_0) = \frac{1}{2}$. We will use this to *estimate* the percolation threshold, $q^*$.
* The **steepness** $\lambda$, which controls how quickly the transition from $p \approx 1$ to $p \approx 0$ occurs.

Have a look at what this function looks like using [Wolfram Alpha](https://www.wolframalpha.com/input/?i=plot+1+%2F+%281+%2B+exp%28lambda+*+%28x+-+0.5%29%29%29++from+0+to+1+varying+lambda+from+10+to+100)

The Logistic function is the simplest function that has essentially the correct shape "backwards S" shape to describe the percolation transition, and since we don't have any other clues (e.g. from an underlying theory), it is a good function to start with in this case.
It is very important to realise that we do not have any fundamental reason to think $S(q)$ is going to fit the data perfectly, but we have to start somewhere!

Run the cell below, which will measure $f$ over a range of $q$, plot the best-fit logistic function, and print the best-fit parameters.

<div class="alert alert-block alert-warning">
<b>Warning!</b> 
    Notice that the program has printed its estimates for $q_0$ and $\lambda$ with a <b>very high precision</b>.
    When it comes to reporting results in your lab book, you should decide on an appropriate number of significant figures for these values, using the rules you have learnt in the P1B course.
    Hint: the full precision given by the program is <b>not appropriate</b>!
</div>

In [ ]:
lattice = SquareLattice(50, 50, n_links=3)
model = PercolationModel(lattice)

parameter_scan(model, start=0, stop=1, num=25, repeats=25)

What's wrong with this plot?

Well, most of the data points are just zeroes or ones, and we have very few points that are close to the percolation transition at $f \approx 0.5$.
This is a problem because it reduces the accuracy of our determination of $q_0$, and also stops us from really being able to check whether a logistic curve fits the data well or not.

<div class="alert alert-block alert-warning">
<b>Warning!</b> 
    You may be wondering why the error printed by the program seems so small.
    The answer is that the error is an <b>estimate</b> calculated using the data points themselves.
    If there are a very small number of data points, it is more likely that the best-fit curve will be able to pass through all of their error bars, thus making the error appear small.
    Don't be fooled!
    If the error gets smaller as you remove data points, its probably a bad estimate.
</div>

In the cell below, try to find values for the end-points of the plot, `qmin` and `qmax`, so that most of the data points are not equal to zero or one.

In [ ]:
lattice = SquareLattice(50, 50, n_links=3)
model = PercolationModel(lattice)

qmin = 0.4
qmax = 0.43

parameter_scan(model, qmin, qmax, num=25, repeats=25)

## Residuals

After fitting a function, there are a number of tests we can do to check whether or not the function fits the data well.
This is particularly important since we really have no theoretical reason to believe that the data should follow a logistic curve.

We will focus on two very simple but useful ways to check the quality of the fit:

1. Checking that the best-fit curve passes through approximately 2/3 of the error bars.
2. Checking for the absence of **structure in the residuals.**

<div class="alert alert-block alert-success">
    <b>Definition:</b> <b>Residuals</b> are the differences between the data points and the best-fit curve, and can be plotted by choosing the best-fit curve as an x-axis.
    If the residuals show an underlying <b>structure</b> then this implies that the best-fit line is <b>systematically</b> incorrect.
</div>

Residuals do not contain any additional information that isn't already in the main plot, but by choosing the x-axis to be the best-fit line it can be easier to check whether the fit is a good one or not.

Run the cell below to see same examples of residuals with and without obvious underlying structure.

In [ ]:
from utils import residuals_examples
residuals_examples()

When the residuals have an obvious structure, as in the top two figures, this is telling us that our best-fit curve has not completely captured the underlying mechanism that produced the data.

However, you should be careful not to over-interpret any small trends in the residuals.
For example, if five points in a row are all below the best-fit curve, this doesn't necessarily represent underlying structure; it could easily be a statistical fluke.

Run the cell below to perform another scan over the $q$ parameter.

In [ ]:
lattice = SquareLattice(100, 50, n_links=3)
model = PercolationModel(lattice)

qmin = 0.33
qmax = 0.43

parameter_scan(model, qmin, qmax, num=50, repeats=50)

<div class="alert alert-block alert-info">
<b>Lab book: (2.1)</b>
Comment on the quality of the logistic best-fit curve, referring to the error bars and the residuals.
    You should include a copy of the plot in your lab book; right click the figure and either copy-paste it into your lab book, or select `Save Image As`, save it to your device, and then load the saved file into your lab book.
</div>

## Extrapolating to infinite lattice size

There is one glaring issue with all the analysis we have done so far; all of our results depend on the size of the lattice!

So-called **finite-volume effects** are an inevitability with simulations, since we cannot simulate an infinite system.
However, it is possible to *extrapolate* results to the infinite limit, by looking at how the thing you're measuring varies as the size of simulations increases.
The answers you get after extrapolating are more likely to agree with the physical world.

To make things simpler (and quicker), we will leave the number of columns fixed at 50, and just vary the number of rows.
Let's label the number of rows as $L$.

We will attempt to extract some infinite-lattice results for the percolation threshold, $\lim_{L\to\infty} q^*$.

<div class="alert alert-block alert-info">
<b>Lab book: (Table 2)</b> 
By varying `n_rows` ($L$), `qmin` and `qmax`, and repeatedly running the cell below, obtain estimates of $q^*$ for for 5-10 different lattice sizes ranging between `n_rows = 50` and `n_rows = 400`. 
Record your results in a table, making sure to note down the associated errors on $q_0$ <b>with an appropriate precision</b>.
Also record your values for `qmin` and `qmax` in the table; these will need to be carefully selected for each lattice size so that almost all of the data points take values between 0 and 1, but are not 0 or 1.
</div>


<div class="alert alert-block alert-warning">
<b>Warning!</b> 
    The larger simulations will take a while to complete. There is no need to generate more than 25 data points for each lattice size, and 25 repeats should also suffice. The largest lattice size may take up to 5 minutes, so grab a cup of coffee while you wait!
</div>

In [ ]:
n_rows = 50  # this is L
qmin = 0.3
qmax = 0.5

lattice = SquareLattice(n_rows, 50, n_links=3)
model = PercolationModel(lattice)

parameter_scan(model, qmin, qmax, num=25, repeats=25)

You may be able to tell, by just studying the numbers in your table, that the relationship between lattice size (`n_rows`) and the mid-point of the transition is not linear.

We really can't say *a priori* what the relationship is, but it can be a idea to take logarithms to check if the relationship is a power-law $y \propto x^a$.


<div class="alert alert-block alert-info">
<b>Lab book: (Table 2)</b> 
In the remaining three columns of your table, calculate $\ln L$, $\ln q_0$, and the error on $\ln q_0$.
</div>

This will require you to *propagate* the error on $q_0$ through the logarithm.
Refer to the lab manual if you need to double check how to do this.

<div class="alert alert-block alert-info">
<b>Lab book: (Plot 1)</b> 
Plot a graph of $\ln L$ v.s. $\ln q_0$ and draw a line of best fit.
Calculate the gradient of your line, with an estimate of its error.
</div>

Now, let's say we take the lattice size to tend towards infinity (i.e. we move right-wards along the horizontal axis).

<div class="alert alert-block alert-info">
<b>Lab book: (2.4)</b>
    It can be argued from a theoretical perspective that the percolation threshold $q^*$ will tend to zero as the number of rows tends to infinity: $\lim_{L \to \infty} q^* = 0$.
    Is your plot consistent with this hypothesis?
</div>

<div class="alert alert-block alert-info">
<b>Lab book: (2.5)</b>
    Can you explain why we might expect $\lim_{L \to \infty} q^* = 0$ ?
</div>

**Hint** for 2.5: Consider the probability that the water succeeds in percolating through a fixed number of rows, $l$.
This will be a fixed number $p_l < 0$ provided $q > 0$.
Use one of the rules of probability from the previous notebook to write down the probability that the water percolates through $2l$ rows, and then extend this to the case of $kl$ rows where $k \to \infty$.

<div class="alert alert-block alert-danger">
<b>Notebook complete:</b> You're ready to move on to Notebook 3.
</div>

# Further reading: Fitting functions to data

<div class="alert alert-block alert-warning">
<b>Warning!</b> 
    Instead of talking of fitting <b>functions</b> to data, it is more common to speak of fitting a <b>model</b> to some observations.
    In this situation, a model is just a construct which, through the correct choice of its parameters, purports to be able to predict the observations.
</div>

Throughout your time at school or college you may have spent a good deal of time drawing straight lines through data on paper, or perhaps even using something like Excel.

*Linear fits* are exceptionally useful, and as such, if it is possible to manipulate your data into a form where it is fit well by a straight line, then this is the approach you should take.
As an example, we frequently take the logarithm of our variables if they obey a power law $y = A x^\alpha$, since the variables $\log y$ and $\log x$ are related by the *linear* equation $\log y = \alpha \log x + \log A$, which is much easier to work with.

However, it is not always possible to find a form in which our data is fit well by a straight line.
We might imagine that there are more suitable functions that fit our data better.
For example, if we were measuring the voltage of mains AC as a function of time, this is likely to be fit well by a sinusoid $V(t) = A \sin\big( (100\pi~\mathrm{s}^{-1}) t + \varphi\big)$, where we can tune the amplitude $A$ and the phase shift $\varphi$ (the frequency we know is 50Hz) to get the best fit.
In contrast, there is no way we can tune the gradient $m$ and the y-intercept $c$ to make the straight line $V(t) = m t + c$ fit the data well!

But drawing a general function by hand is very difficult!
How do we *actually do the fit*?
This is a very complicated question, but all you need to understand at this point is that there are *algorithms* that *automatically* tune the *free parameters* of your function so that it fits the data as well as possible.

The most important of these is called the **method of least squares**.
We can write a computer algorithm that performs *least-squares fitting* by repeatedly 'nudging' the free parameters, so that each nudge results in a reduction in a quantity which encapsulates how well the function fits the data.
For the curious, this quantity is result of adding up the *squared differences* between each data point and the function that you are trying to fit.
This is the square of the residuals that we've been plotting.

However, with such power comes, as always, great responsibility.
The famous mathematician **John von Neumann** once said

> "With four free parameters I can fit an elephant, and with five I can make him wiggle his trunk!"

What he means is, you could come up with an extremely complicated function, with many many free parameters, and the least squares algorithm could twiddle these parameters so that the function passed through all of the error bars on your data points (which could be in the shape of an elephant!)
This *does not mean that your function is a good representation of the underlying laws that generated the data*.

If you're struggling to follow this, don't worry!
Just bear it in mind as you continue your degree:
**If two functions fit your data equally well, and one has fewer free parameters than the other, then you should choose the one with fewer parameters.**
This is a version of **Occam's razor**, which basically says that "the simplest theory that accurately predicts the phenomena is usually the correct one".


